In [1]:
if True:
    import pandas as pd
    from datasets import Dataset
    from transformers import DistilBertTokenizer

    X_train = pd.read_csv("../data/Train_Test_splits/X_train_50proc_trunc_pad.csv")
    X_test = pd.read_csv("../data/Train_Test_splits/X_test_50proc_trunc_pad.csv")
    y_train = pd.read_csv("../data/Train_Test_splits/y_train_50proc.csv")
    y_test = pd.read_csv("../data/Train_Test_splits/y_test_50proc.csv")

    print(X_train.columns)
    print(X_test.columns)
    print(y_train.columns)
    print(y_test.columns)

/home/buntish/.local/share/virtualenvs/Test_DL-u3DERucF/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['reviews_decoded'], dtype='object')
Index(['reviews_decoded'], dtype='object')
Index(['sentiment'], dtype='object')
Index(['sentiment'], dtype='object')


# TF #

In [2]:
if False:
    from datasets import load_dataset
    from transformers import TFAutoModelForSequenceClassification, DistilBertTokenizer
    import tensorflow as tf
    from sklearn.metrics import accuracy_score
    import numpy as np

    if not tf.config.list_physical_devices("GPU"):
        print("Warning: No GPU found. Performance may be slower.")

    tokenizer = DistilBertTokenizer.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    )
    model = TFAutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    )

    dataset = load_dataset("imdb")
    test_dataset = dataset["test"].select(range(100))

    def tokenize_function(examples):
        return tokenizer(
            examples["text"], padding="max_length", truncation=True, max_length=512
        )

    test_dataset = test_dataset.map(tokenize_function, batched=True)

    test_tf_dataset = tf.data.Dataset.from_tensor_slices(
        (
            np.array(test_dataset["input_ids"]),
            np.array(test_dataset["attention_mask"]),
            np.array(test_dataset["label"]),
        )
    ).batch(8)

    @tf.function
    def predict_step(input_ids, attention_mask):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
        logits = model(inputs, training=False).logits
        return np.argmax(logits.numpy(), axis=1)

    predictions = []
    labels = []

    for input_ids, attention_mask, batch_labels in test_tf_dataset:
        preds = predict_step(input_ids, attention_mask)
        predictions.extend(preds)
        labels.extend(batch_labels.numpy())

    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy: {accuracy}")

In [3]:
if True:
    import tensorflow as tf
    print(tf.config.list_physical_devices("GPU"))

2025-01-14 09:51:09.783069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736844669.804696    9993 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736844669.810778    9993 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 09:51:09.833534: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


W0000 00:00:1736844672.103257    9993 gpu_device.cc:2433] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.2. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


# Torch #

In [4]:
if True:
    from datasets import load_dataset
    from transformers import AutoModelForSequenceClassification, DistilBertTokenizer
    import torch
    from sklearn.metrics import accuracy_score
    import numpy as np

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = DistilBertTokenizer.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    ).to(device)

    dataset = load_dataset("imdb")
    test_dataset = dataset["test"].select(range(100))

    def tokenize_function(examples):
        return tokenizer(
            examples["text"], padding="max_length", truncation=True, max_length=512
        )

    test_dataset = test_dataset.map(tokenize_function, batched=True)

    inputs = torch.tensor(test_dataset["input_ids"]).to(device)
    attention_mask = torch.tensor(test_dataset["attention_mask"]).to(device)
    labels = torch.tensor(test_dataset["label"]).to(device)

    predictions = []
    true_labels = []

    with torch.no_grad():
        for i in range(0, len(inputs), 8):
            batch_input_ids = inputs[i : i + 8]
            batch_attention_mask = attention_mask[i : i + 8]
            batch_labels = labels[i : i + 8]
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch_labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy}")

Accuracy: 0.89


In [5]:
if True:
    import tensorflow as tf

    print(tf.config.list_physical_devices("GPU"))
    devices = tf.config.list_physical_devices()
    print(devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


: 

In [ ]:
if True:
    import tensorflow as tf

    device = tf.config.list_physical_devices("GPU")

    device_string = "GPU is available!" if device else "GPU is NOT available."
    print()
    print(device_string)
    print()

    # Somehow this takes unbelievably long time...
    with tf.device("/GPU:0"):
        a = tf.random.normal([10, 10])
        b = tf.random.normal([10, 10])
        c = tf.matmul(a, b)

    print("Matrix multiplication done on GPU!")


GPU is available!



W0000 00:00:1736844684.224917    9993 gpu_device.cc:2433] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.2. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1736844684.225387    9993 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 491 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 960, pci bus id: 0000:01:00.0, compute capability: 5.2
F0000 00:00:1736844684.262729    9993 random_op_gpu.h:247] Non-OK-status: GpuLaunchKernel(FillPhiloxRandomKernelLaunch<Distribution>, num_blocks, block_size, 0, d.stream(), key, counter, gen, data, size, dist)
Status: INTERNAL: no kernel image is available for execution on the device
